In [1]:
import numpy as np
import pandas as pd

### Sessions

In [2]:
sessions = pd.read_csv("data/sessions.csv")
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [3]:
sessions = sessions.drop(["device_type"], axis=1)

action_type_counts = sessions.drop(["action", "action_detail"], axis=1)
action_type_counts = action_type_counts.pivot_table(values="action_type", 
                                                    columns="action_type", 
                                                    index="user_id", 
                                                    aggfunc="count").reset_index().fillna(0)

hours_elapsed_sum = (sessions.groupby("user_id").sum()["secs_elapsed"] / 3600).to_frame()
hours_elapsed_sum = hours_elapsed_sum.rename(columns={"secs_elapsed": "hours_elapsed_sum"}).reset_index()

In [4]:
sessions_processed = action_type_counts.merge(hours_elapsed_sum, how="inner", on="user_id")
cols_modif = sessions_processed.columns[1:-1]
sessions_processed[cols_modif] = sessions_processed[cols_modif].astype(int)
sessions_processed["hours_elapsed_sum"] = sessions_processed["hours_elapsed_sum"].apply(lambda x: round(x, 2))

In [5]:
display(sessions_processed.head())
sessions_processed.info()

,user_id,-unknown-,booking_request,booking_response,click,data,message_post,modify,partner_callback,submit,view,hours_elapsed_sum
0,00023iyk9l,0,1,0,4,9,1,0,0,0,21,241.08
1,0010k6l0om,5,0,0,16,9,0,0,0,0,17,162.93
2,001wyh0pz8,6,0,0,66,2,0,0,0,2,8,78.60
3,0028jgx1x1,1,0,0,9,5,0,0,0,0,15,82.50
4,002qnbzfs5,184,1,0,140,140,16,0,0,15,216,1801.97


<class 'pandas.core.frame.DataFrame'>
Int64Index: 135478 entries, 0 to 135477
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            135478 non-null  object 
 1   -unknown-          135478 non-null  int64  
 2   booking_request    135478 non-null  int64  
 3   booking_response   135478 non-null  int64  
 4   click              135478 non-null  int64  
 5   data               135478 non-null  int64  
 6   message_post       135478 non-null  int64  
 7   modify             135478 non-null  int64  
 8   partner_callback   135478 non-null  int64  
 9   submit             135478 non-null  int64  
 10  view               135478 non-null  int64  
 11  hours_elapsed_sum  135478 non-null  float64
dtypes: float64(1), int64(10), object(1)
memory usage: 13.4+ MB


### Main data

In [6]:
main_data = pd.read_csv("data/train_users_2.csv")
main_data.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [7]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       213451 non-null  object 
 1   date_account_created     213451 non-null  object 
 2   timestamp_first_active   213451 non-null  int64  
 3   date_first_booking       88908 non-null   object 
 4   gender                   213451 non-null  object 
 5   age                      125461 non-null  float64
 6   signup_method            213451 non-null  object 
 7   signup_flow              213451 non-null  int64  
 8   language                 213451 non-null  object 
 9   affiliate_channel        213451 non-null  object 
 10  affiliate_provider       213451 non-null  object 
 11  first_affiliate_tracked  207386 non-null  object 
 12  signup_app               213451 non-null  object 
 13  first_device_type        213451 non-null  object 
 14  firs

In [8]:
int_time_to_str = lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:8] + ' ' + x[8:10] + ':' + x[10:12] + ':' + x[12:]

main_data["date_account_created"] = main_data["date_account_created"].astype("datetime64[ns]")
main_data["timestamp_first_active"] = main_data["timestamp_first_active"].astype(str) \
                                                                         .apply(int_time_to_str) \
                                                                         .astype("datetime64[ns]")
main_data["date_first_booking"] = main_data["date_first_booking"].astype("datetime64[ns]")
main_data["signup_flow"] = main_data["signup_flow"].astype(str)

# Dealing with NaN values and clearing dataset

main_data = main_data[(main_data["age"] < 100) | (main_data["age"].isnull())]
main_data = main_data.dropna(subset=["first_affiliate_tracked"])
main_data = main_data.drop("date_first_booking", axis=1)

ages = main_data.groupby(["gender", "country_destination"]).mean().reset_index()
ages = ages.rename(columns={"age": "mean_age"})
ages["mean_age"] = ages["mean_age"].round()
main_data = main_data.merge(ages, on=["gender", "country_destination"])
main_data["age"] = main_data["age"].fillna(main_data["mean_age"])
cleared_data = main_data.drop("mean_age", axis=1)

In [9]:
display(cleared_data.head())
cleared_data.info()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,37.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,jha93x042q,2010-01-11,2010-01-11 22:40:15,-unknown-,37.0,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,NDF
2,bjg0m5otl3,2010-01-12,2010-01-12 15:54:20,-unknown-,37.0,basic,0,en,other,other,untracked,Web,Other/Unknown,-unknown-,NDF
3,mo2v5h3nti,2010-01-14,2010-01-14 08:49:26,-unknown-,31.0,basic,0,en,seo,google,untracked,Web,Other/Unknown,-unknown-,NDF
4,7yzz3iv0ji,2010-01-15,2010-01-15 23:41:32,-unknown-,37.0,basic,0,en,sem-non-brand,other,omg,Web,Windows Desktop,Firefox,NDF


<class 'pandas.core.frame.DataFrame'>
Int64Index: 205065 entries, 0 to 205064
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   id                       205065 non-null  object        
 1   date_account_created     205065 non-null  datetime64[ns]
 2   timestamp_first_active   205065 non-null  datetime64[ns]
 3   gender                   205065 non-null  object        
 4   age                      205065 non-null  float64       
 5   signup_method            205065 non-null  object        
 6   signup_flow              205065 non-null  object        
 7   language                 205065 non-null  object        
 8   affiliate_channel        205065 non-null  object        
 9   affiliate_provider       205065 non-null  object        
 10  first_affiliate_tracked  205065 non-null  object        
 11  signup_app               205065 non-null  object        
 12  first_device_typ

In [10]:
sessions_processed.to_csv("data/sessions_processed.csv", index=False)
cleared_data.to_csv("data/train_users_processed.csv", index=False)